# This notebook prepares the dataset for the fine-tuning process. It consists of combining 4 different dataset and creating and analysing their features including the categories "dataset", "target word" and "gold label".

Importing required libraries and modules.

In [ ]:
import pandas as pd

Loading the first dataset, Kurrek et al's (2020) Slur Corpus.

In [ ]:
dataset= pd.read_csv('Notebook_6_14_kurrek.2020.slur-corpus.csv')

Displaying the dataframe.

In [ ]:
dataset

,id,link_id,parent_id,score,subreddit,author,slur,body,disagreement,gold_label
0,t1_c3s59yl,t3_pta2s,t1_c3s22n6,10.0,funny,AntiHer0z,tranny,Fuck that I dont wanna watch tranny porn.,False,DEG
1,t1_e5li3m3,t3_9e0lfg,t3_9e0lfg,34.0,opieandanthony,cumspaces,tranny,Opie just wanted to have a good time. The same...,False,DEG
2,t1_e5tpq0a,t3_9f3ytm,t3_9f3ytm,61.0,opieandanthony,SchumersStolenJokes,faggot,Fuck that faggot Fallon. Fucking sissy boy act...,False,DEG
3,t1_c93xbko,t3_1b5k0c,t1_c93wllg,605.0,pics,notarapist72,faggot,BbBB...b.b..b.bb but OP's a faggot,False,DEG
4,t1_e1wwfj1,t3_8wj0bg,t1_e1wvjyg,8.0,Guildwars2,drasticpear,tranny,Who even uses the word tranny except for trans...,False,NDG
...,...,...,...,...,...,...,...,...,...,...
39998,t1_chr083m,t3_26gq07,t1_chr04p7,-104.0,pics,espionage101,faggot,*As an alcoholic who on occasion has one too m...,False,DEG
39999,t1_ds0k90g,t3_7nbdby,t1_ds0ijht,63.0,WhiteRights,hvvhnuifihnsefvhnuis,nigger,"It's not inaccurate, it's indoctrination of yo...",False,DEG
40000,t1_cb2wbf5,t3_1ian9r,t1_cb2t16t,63.0,pics,gooby_no_pls,nigger,What did you expect?? when they banned /r/nigg...,True,NDG
40001,t1_cb6u9kx,t3_1ip6x6,t1_cb6rkxi,49.0,AskReddit,MeVasta,tranny,That was hilariously bad. They used the insult...,False,NDG


Displaying value counts for the slurs.

In [ ]:
dataset['slur'].value_counts()

tranny    13334
nigger    13334
faggot    13332
Name: slur, dtype: int64

Selecting the target slur.

In [ ]:
lgbt_slur = dataset[dataset['slur'] == 'faggot']

Displaying value counts for the gold_label category of the target slur.

In [ ]:
value_counts = lgbt_slur['gold_label'].value_counts(normalize=True) * 100
print(value_counts)

DEG    58.888389
NDG    38.681368
APR     1.522652
HOM     0.592559
CMP     0.315032
Name: gold_label, dtype: float64


Applying hate labels to the target word where label is 1 (hate) if gold_label is DEG and 0 for all other categories.

In [ ]:
lgbt_slur = lgbt_slur.copy()  # Create a copy of the DataFrame
lgbt_slur['hate_label'] = lgbt_slur['gold_label'].apply(lambda x: 1 if x == 'DEG' else 0)

In [ ]:
lgbt_slur = lgbt_slur.rename(columns={'body': 'sentences'})

Adding the dataset column.

In [ ]:
lgbt_slur['dataset'] = 'kurrek'

Loading the second dataset, the scraped Drag Race transcript sentences which were predicted hateful by Vidgen et al's model.

In [ ]:
drag_transcript = pd.read_csv('Notebook_6_7_dragrace_transcript_wrongpreds.csv')

Defining the hate labels (0 = nonhate) and gold label columns for the Drag Race transcript

In [ ]:
drag_transcript['hate_label'] = 0
drag_transcript['gold_label'] = 'APR'

Removing symbols.

In [ ]:
drag_transcript['sentences'] = drag_transcript['sentences'].apply(lambda x: x.replace("♪", ""))

Defining the dataset column in the drag race transcript dataset.

In [ ]:
drag_transcript['dataset'] = 'drag_transcript'

Loading the third dataset, the synthetically generated hate speech and reclaimed language dataset.

In [ ]:
synth_data = pd.read_csv('Notebook_6_Synthetic_Dataset.csv')

Renaming columns to be consistent with other datasets.

In [ ]:
synth_data = synth_data.rename(columns={'Sentence': 'sentences', 'Label': 'hate_label'})

Defining the gold labels for each category of the dataset.

In [ ]:
synth_data['gold_label'] = None
synth_data.loc[:49, 'gold_label'] = 'DEG'
synth_data.loc[50:94, 'gold_label'] = 'APR'
synth_data.loc[95:111, 'gold_label'] = 'APR_negexp'
synth_data.loc[112:153, 'gold_label'] = 'APR_negimp'
synth_data['dataset'] = 'synth'

Loading the fourth dataset, the Kaggle Hate Speech Dataset (Samoshyn, 2020)

In [ ]:
kaggle_hatespeech = pd.read_csv('Notebook_6_HateSpeechandOffensiveLanguageDataset_kaggle.csv')

Identifying sentences in the dataset with chosen target words.

In [ ]:
# Creating a pattern to match the key words with variations

pattern = r'\b(?:bitch|drag|fag|gay|homo|puss|queen|queer|sissy|slut|whore)\w*\b'

# Using the pattern to filter rows
kaggle_targets = kaggle_hatespeech[kaggle_hatespeech['tweet'].str.contains(pattern, case=False, regex=True)]


Defining the gold label, hate label and dataset columns for the dataset.

In [ ]:
import pandas as pd

# Assuming you have a DataFrame named kaggle_targets

# Filter rows with 'hate_speech' equal to 1 and create a copy
kaggle_targets_hate = kaggle_targets[kaggle_targets['hate_speech'] == 1].copy()
kaggle_targets_hate['gold_label'] = 'DEG'

# Filter rows with 'neither' equal to 1 and create a copy
kaggle_targets_nohate = kaggle_targets[kaggle_targets['neither'] == 1].copy()
kaggle_targets_nohate['gold_label'] = 'NDG'

# Concatenate the filtered DataFrames
kaggle = pd.concat([kaggle_targets_nohate, kaggle_targets_hate])

# Rename columns
kaggle = kaggle.rename(columns={'tweet': 'sentences', 'hate_speech': 'hate_label'})

# Add a 'dataset' column
kaggle['dataset'] = 'kaggle'


In [ ]:
kaggle['hate_label'].value_counts()

1    1991
0     497
2      20
3       1
Name: hate_label, dtype: int64

Removing outlying hate labels from the dataset.

In [ ]:
mask = kaggle['hate_label'].isin([2, 3])

# Filter out rows with values 2 and 3 in 'hate_label'
kaggle = kaggle[~mask]

In [ ]:
kaggle['hate_label'].value_counts()

1    1991
0     497
Name: hate_label, dtype: int64

Producing the final dataset for fine tuning by concatenating the required columns from each of the four datasets.

In [ ]:
fine_tune_df = pd.concat([lgbt_slur[['sentences', 'hate_label', 'gold_label', 'dataset']], drag_transcript[['sentences', 'hate_label', 'gold_label', 'dataset']], synth_data[['sentences', 'hate_label', 'gold_label', 'dataset']], kaggle[['sentences', 'hate_label', 'gold_label', 'dataset']]], ignore_index=True)

In [ ]:
import re

Defining binary columns in the dataset to express the presence of each target word (1 = present) in the sentences column.

In [ ]:
import pandas as pd
import string

def target_word(sentence):
    keywords = ['bitch', 'drag', 'fag', 'gay', 'homo', 'puss', 'queen', 'queer', 'sissy','slay', 'slut', 'whore']
    variations = {
        'whore': ["hoe", "ho", "ho's"], # Variations of 'whore'
    }

    found_keywords = set()  # Use a set to store unique keywords
    words = sentence.split()

    # Remove punctuation from words and make them lowercase
    cleaned_words = [word.strip(string.punctuation).lower() for word in words]

    for keyword in keywords:
        if keyword == 'whore':
            if keyword in cleaned_words or any(var in cleaned_words for var in variations.get(keyword, [])):
                found_keywords.add(keyword)
        else:
            if any(keyword in word for word in cleaned_words):
                found_keywords.add(keyword)

    return found_keywords if found_keywords else {'other'}

keywords = ['bitch', 'drag', 'fag', 'gay', 'homo', 'other', 'puss', 'queen', 'queer', 'sissy','slay', 'slut', 'whore']

# Apply the target_word function to create the 'key_words' column
fine_tune_df['key_words'] = fine_tune_df['sentences'].apply(target_word)

# Create new columns for each keyword and populate with binary values
for keyword in keywords:
    fine_tune_df[keyword] = fine_tune_df['key_words'].apply(lambda keywords: 1 if keyword in keywords else 0)

# Drop the 'key_words' column as it's no longer needed
fine_tune_df.drop('key_words', axis=1, inplace=True)

In [ ]:
fine_tune_df = fine_tune_df.drop_duplicates(keep='first')

In [ ]:
fine_tune_df

,sentences,hate_label,gold_label,dataset,bitch,drag,fag,gay,homo,other,puss,queen,queer,sissy,slay,slut,whore
0,Fuck that faggot Fallon. Fucking sissy boy act...,1,DEG,kurrek,0,0,1,0,0,0,0,0,0,1,0,0,0
1,BbBB...b.b..b.bb but OP's a faggot,1,DEG,kurrek,0,0,1,0,0,0,0,0,0,0,0,0,0
2,I'm just happy that anyone wants this faggot b...,1,DEG,kurrek,0,0,1,0,0,0,1,0,0,0,0,0,0
3,Faggot \nJk. \nIt looks awesome!,1,DEG,kurrek,0,0,1,0,0,0,0,0,0,0,0,0,0
4,That's like saying the the n word belong to hi...,0,NDG,kurrek,0,0,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17001,why tf is this bitch in my class wearing all b...,1,DEG,kaggle,1,0,0,0,0,0,0,0,0,0,0,0,0
17002,wow! RT @Tendilkar: RT @LyricalSunset: RT @Lil...,1,DEG,kaggle,1,0,0,0,0,0,0,0,0,0,0,0,0
17003,yal some faggots yup nigga you n you fuckin fr...,1,DEG,kaggle,0,0,1,0,0,0,0,0,0,0,0,0,0
17004,you ain't a real nigga without goal a to chase...,1,DEG,kaggle,1,0,0,0,0,0,0,0,0,0,0,0,0


Removing all sentences containing the word that will be used for the Unseen Word dataset ("tranny" and its variants)

In [ ]:
# Use str.contains() to find sentences containing the specific word
keywords = ["tranny", "trannies", "trannie", "Tranny" ]

fine_tune_df['clean_sentences'] = fine_tune_df['sentences'].str.lower()

# Create a pattern that matches any of the keywords
pattern = '|'.join(keywords)

# Find rows containing any of the keywords in the 'text' column
matching_rows = fine_tune_df[fine_tune_df['clean_sentences'].str.contains(pattern, case=False)]

# Print the result
print(len(matching_rows))

15


<ipython-input-71-b7eecde3141b>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Drop 15 rows which conatin "Tranny" or variations
fine_tune_df = fine_tune_df[~fine_tune_df['clean_sentences'].str.contains(pattern, case=False)]

Analysing distribution of target words in final dataset.

In [ ]:
# Print value counts with value 1 for each column
keywords = ['bitch', 'drag', 'fag', 'gay', 'homo', 'other', 'puss', 'queen', 'queer', 'sissy','slay', 'slut', 'whore']
values = []
labels = []

for keyword in keywords:
    value_count = fine_tune_df[keyword].sum()
    values.append(value_count)
    labels.append(keyword)
    print(f"Value count for '{keyword}' with value 1: {value_count}")

Value count for 'bitch' with value 1: 2560
Value count for 'drag' with value 1: 166
Value count for 'fag' with value 1: 13737
Value count for 'gay' with value 1: 1698
Value count for 'homo' with value 1: 586
Value count for 'other' with value 1: 381
Value count for 'puss' with value 1: 592
Value count for 'queen' with value 1: 96
Value count for 'queer' with value 1: 218
Value count for 'sissy' with value 1: 29
Value count for 'slay' with value 1: 12
Value count for 'slut' with value 1: 56
Value count for 'whore' with value 1: 138


In [ ]:
# Calculate the total sum of values
total = sum(values)

# Calculate the percentage for each category
percentages = [value / total * 100 for value in values]

# Create a table using a pandas DataFrame
table = pd.DataFrame({
    'Label': labels,
    'Percentage': percentages
})

# Sort the table by the "Percentage" column in descending order
table = table.sort_values(by='Percentage', ascending=False)

# Display the sorted table
print(table)


    Label  Percentage
2     fag   67.773447
0   bitch   12.630125
3     gay    8.377325
6    puss    2.920716
4    homo    2.891115
5   other    1.879718
8   queer    1.075534
1    drag    0.818985
12  whore    0.680843
7   queen    0.473630
11   slut    0.276284
9   sissy    0.143076
10   slay    0.059204


Visualisation of target word proportions.

In [ ]:
import plotly.express as px

# Define the color sequence using the Viridis palette
color_palette = px.colors.sequential.haline[::-1]

# Create the pie chart with the reversed color palette
fig = px.pie(values=values, names=labels, title='Distribution of Keywords',
             color_discrete_sequence=color_palette)

# Center the title and set its text
fig.update_layout(title={'text': "Distribution of Keywords", 'x': 0.5, 'xanchor': 'center'})

# Show the plot
fig.show()

Downloading CSV file

In [ ]:
from google.colab import files

# Assuming that df is your DataFrame
fine_tune_df.to_csv('fine_tune_final.csv', index=False)

# Now download the csv file locally
files.download('fine_tune_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Analysing distributions of dataset characteristics.

In [ ]:
fine_tune_df['dataset'].value_counts(normalize = True) *100

kurrek             78.788238
kaggle             14.720180
drag_transcript     5.578610
synth               0.912971
Name: dataset, dtype: float64

In [ ]:
fine_tune_df['gold_label'].value_counts(normalize = True) *100

DEG           58.353095
NDG           33.542803
APR            7.042922
HOM            0.468342
APR_negimp     0.248992
CMP            0.243064
APR_negexp     0.100783
Name: gold_label, dtype: float64

In [ ]:
# Defining the columns to analyze
columns_to_analyze = fine_tune_df.columns[4:17]

# Function to calculate the value counts where the value is 1
def value_counts_for_1(group):
    return group[group == 1].count()

# Grouping by the "hate_label" column and applying the function to the specified columns
grouped_data = fine_tune_df.groupby('hate_label')[columns_to_analyze].apply(value_counts_for_1).reset_index()

# Displaying the result
grouped_data

,hate_label,bitch,drag,fag,gay,homo,other,puss,queen,queer,sissy,slay,slut,whore
0,0,892,144,5588,1209,469,380,164,64,153,16,10,25,51
1,1,1668,22,8149,489,117,1,428,32,65,13,2,31,87


Analysing proportions of each target word that is hateful vs non hateful.

In [ ]:
# Define the columns to analyze
columns_to_analyze = fine_tune_df.columns[4:17]

# Function to calculate the value counts where the value is 1
def value_counts_for_1(group):
    return group[group == 1].count()

# Group by the "hate_label" column and apply the function to the specified columns
grouped_data = fine_tune_df.groupby('hate_label')[columns_to_analyze].apply(value_counts_for_1).reset_index()

# Calculate the percentage for each value count by dividing by the sum of each column and multiplying by 100
percentage_data_relative = grouped_data.copy()
percentage_data_relative[columns_to_analyze] = grouped_data[columns_to_analyze].div(grouped_data[columns_to_analyze].sum(), axis=1) * 100

# Display the result as percentages relative to each column
print(percentage_data_relative)


   hate_label     bitch       drag       fag        gay      homo      other  \
0           0  34.84375  86.746988  40.67846  71.201413  80.03413  99.737533   
1           1  65.15625  13.253012  59.32154  28.798587  19.96587   0.262467   

        puss      queen      queer      sissy       slay       slut      whore  
0  27.702703  66.666667  70.183486  55.172414  83.333333  44.642857  36.956522  
1  72.297297  33.333333  29.816514  44.827586  16.666667  55.357143  63.043478  


Analysing gold label distribution of target words.

In [ ]:
# Grouping by the "gold_label" column and applying the function to the specified columns
grouped_data_gold_label = fine_tune_df.groupby('gold_label')[columns_to_analyze].apply(value_counts_for_1).reset_index()

# Calculating the percentage for each value count by dividing by the sum of each column and multiplying by 100
percentage_data_gold_label = grouped_data_gold_label.copy()
percentage_data_gold_label[columns_to_analyze] = grouped_data_gold_label[columns_to_analyze].div(grouped_data_gold_label[columns_to_analyze].sum(), axis=1) * 100

# Displaying the result as percentages relative to each column, grouped by the "gold_label" column
percentage_data_gold_label


,gold_label,bitch,drag,fag,gay,homo,other,puss,queen,queer,sissy,slay,slut,whore
0,APR,14.68750,74.698795,1.798064,5.182568,1.877133,99.737533,3.716216,47.916667,6.422018,3.448276,25.000000,8.928571,18.115942
1,APR_negexp,0.00000,0.000000,0.094635,0.588928,0.000000,0.000000,0.000000,0.000000,1.834862,0.000000,0.000000,0.000000,0.000000
2,APR_negimp,0.00000,0.000000,0.305744,0.000000,0.000000,0.000000,0.000000,7.291667,0.000000,0.000000,41.666667,0.000000,0.000000
3,CMP,0.00000,0.000000,0.298464,0.176678,0.170648,0.000000,0.168919,0.000000,0.000000,0.000000,0.000000,0.000000,0.724638
4,DEG,64.84375,13.253012,59.306981,28.798587,19.965870,0.262467,71.959459,33.333333,29.816514,44.827586,16.666667,55.357143,63.043478
5,HOM,0.00000,0.602410,0.575089,0.471143,0.511945,0.000000,0.000000,0.000000,0.458716,0.000000,0.000000,0.000000,0.000000
6,NDG,20.46875,11.445783,37.621024,64.782097,77.474403,0.000000,24.155405,11.458333,61.467890,51.724138,16.666667,35.714286,18.115942


In [ ]:
# Define the columns to analyze
columns_to_analyze = fine_tune_df.columns[4:17]

# Function to calculate the value counts where the value is 1
def value_counts_for_1(group):
    return group[group == 1].count()

# Group by the "gold_label" column and apply the function to the specified columns
grouped_data_gold_label = fine_tune_df.groupby('gold_label')[columns_to_analyze].apply(value_counts_for_1).reset_index()

# Define the APR categories to combine
apr_categories = ['APR', 'APR_negimp', 'APR_negexp']

# Sum the rows corresponding to the "APR" categories
apr_all_row = grouped_data_gold_label[grouped_data_gold_label['gold_label'].isin(apr_categories)].sum(numeric_only=True)
apr_all_row['gold_label'] = 'APR_all'

# Append the "APR_all" row to the DataFrame and keep the other rows except the original "APR" categories
grouped_data_with_apr_all = grouped_data_gold_label[~grouped_data_gold_label['gold_label'].isin(apr_categories)].append(apr_all_row, ignore_index=True)

# Calculate the percentage for each value count by dividing by the sum of each column and multiplying by 100
percentage_data_with_apr_all_corrected = grouped_data_with_apr_all.copy()
percentage_data_with_apr_all_corrected[columns_to_analyze] = grouped_data_with_apr_all[columns_to_analyze].div(grouped_data_with_apr_all[columns_to_analyze].sum(), axis=1) * 100

# Display the result as percentages relative to each column, grouped by the "gold_label" column, with the newly created "APR_all" group
print(percentage_data_with_apr_all_corrected)



  gold_label     bitch       drag        fag        gay       homo      other  \
0        CMP   0.00000   0.000000   0.298464   0.176678   0.170648   0.000000   
1        DEG  64.84375  13.253012  59.306981  28.798587  19.965870   0.262467   
2        HOM   0.00000   0.602410   0.575089   0.471143   0.511945   0.000000   
3        NDG  20.46875  11.445783  37.621024  64.782097  77.474403   0.000000   
4    APR_all  14.68750  74.698795   2.198442   5.771496   1.877133  99.737533   

        puss      queen      queer      sissy       slay       slut      whore  
0   0.168919   0.000000   0.000000   0.000000   0.000000   0.000000   0.724638  
1  71.959459  33.333333  29.816514  44.827586  16.666667  55.357143  63.043478  
2   0.000000   0.000000   0.458716   0.000000   0.000000   0.000000   0.000000  
3  24.155405  11.458333  61.467890  51.724138  16.666667  35.714286  18.115942  
4   3.716216  55.208333   8.256881   3.448276  66.666667   8.928571  18.115942  


<ipython-input-84-23f610d48548>:19: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

